***Building DCNN for Facial Emotion Recognition (FER)***

In [1]:
import math
import numpy as np
import pandas as pd
import scikitplot
import seaborn as sns
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

2021-10-28 18:33:48.932503: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-28 18:33:48.932564: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# leitura dos dados do FER2013
df = pd.read_csv('../input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv')
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv'

In [3]:
df.shape

NameError: name 'df' is not defined

In [ ]:
img_array = df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img_array = np.stack(img_array, axis=0)

In [ ]:
img_array.shape

In [ ]:
df.emotion

In [ ]:
le = LabelEncoder()
img_labels = le.fit_transform(df.emotion)
img_labels = np_utils.to_categorical(img_labels)
img_labels.shape

In [ ]:
img_labels[1]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    img_array, img_labels,
    shuffle=True, stratify=img_labels,
    test_size=0.1, random_state=42)

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]

In [ ]:
# Normalizing results, as neural networks are very sensitive to unnormalized data.
X_train = X_train / 255.
X_valid = X_valid / 255.

In [ ]:
# Inicializing the cnn
cnn = Sequential()

# First Layer 
cnn.add(
    Conv2D(
        filters = 64,
        kernel_size=(5,5),
        input_shape=(img_width, img_height, img_depth),
        activation = 'elu',
        name = 'Conv2D_1'
    )
)

cnn.add(BatchNormalization(name='batchnorm_1'))
    
# Second Layer
cnn.add(
        Conv2D(
            filters=64,
            kernel_size=(5,5),
            activation='elu',
            name='conv2d_2'
        )
    )

cnn.add(BatchNormalization(name='batchnorm_2'))

cnn.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))
cnn.add(Dropout(0.4, name='dropout_1'))

# Third Layer
cnn.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            name='conv2d_3'
        )
    )

cnn.add(BatchNormalization(name='batchnorm_3'))
    
# Fouth Layer
cnn.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            name='conv2d_4'
        )
    )
cnn.add(BatchNormalization(name='batchnorm_4'))

cnn.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'))
cnn.add(Dropout(0.4, name='dropout_2'))


# flattening
cnn.add(Flatten(name='flatten'))
    
# densing
cnn.add(
    Dense(
        128,
        activation='elu',
        kernel_initializer='he_normal',
        name='dense_1'
    )
)
cnn.add(BatchNormalization(name='batchnorm_7'))

cnn.add(Dropout(0.6, name='dropout_4'))

cnn.add(
    Dense(
        num_classes,
        activation='softmax',
        name='out_layer'
    )
)

cnn.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    

In [ ]:

train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
)


train_datagen.fit(X_train)

In [ ]:
cnn.summary()

In [ ]:
batch_size = 32 #batch size of 32 performs the best.
epochs = 70

In [ ]:
history = cnn.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_valid, y_valid),
    steps_per_epoch=len(X_train) / batch_size,
    epochs=epochs,
    use_multiprocessing=True
)

In [ ]:
cnn.save("MY_CNN_1.h5")